# Packages

The cell below contains the packages that are used in the code.

In [ ]:
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.ModularVisualization import ModularServer
from mesa.batchrunner import BatchRunner
from IconicityModel import *
import pandas as pd
 
# workaround for the tornado 5.0 bug
import nest_asyncio
nest_asyncio.apply()

# Agent representation

This section contains code about the agent representation. Agents will be represented via circles. The colour and whether it is a filled circle or not depends on the properties of the agent.

The radius of the circles indicate the age of the agent. A smaller circle represents an agent of age 0 and a bigger circle represents an agent of age 1. The colour indicates the age of acquisition of the agent; green stands for L1 signers and red for L2 signers.

In [ ]:
def agent_representation(agent):
    iconicity_degree = round(agent.average_iconicity_degree())
    representation = {
                      # shape and colour of the agent
                      "Shape": "circle",
                      "Filled": "true",
                      "r": 0.5,
                      
                      # visible properties in visualisation
                      "Age of acquisition": agent.aoa,
                      "Age": agent.age,
                      "Avg. iconicity level": str(iconicity_degree) + "%",
        
                      # drawing layer
                      "Layer": 0,
                      }
    
    if agent.age == 0:
        representation["r"] = 0.3
    elif agent.age == 1:
        representation["r"] = 0.5
        
    if agent.aoa == "L1":
        representation["Color"] = green_colour(iconicity_degree)
    elif agent.aoa == "L2":
        representation["Color"] = red_colour(iconicity_degree)
                                             
    return representation
    
def green_colour(iconicity_degree):
    if iconicity_degree <= 33:
        return "#d9ffbf"
    elif iconicity_degree <= 66:
        return "#85cc6f"
    elif iconicity_degree <= 100:
        return "#136207"

def red_colour(iconicity_degree):
    if iconicity_degree <= 33:
        return "#d19999"
    elif iconicity_degree <= 66:
        return "#ae4d4d"
    elif iconicity_degree <= 100:
        return "#8b0000"

# Grid representation

This section contains code about the grid representation.

In [ ]:
grid = CanvasGrid(agent_representation, 10, 10, 500, 500)

# Server representation

This section contains code about the server representation.

In [ ]:
# Word length
initial_word_length = 5
maximal_word_length = 10
minimal_word_length = 5

# Vocabulary size
initial_vocab_size = 5
maximal_vocab_size = 2 ** maximal_word_length
minimal_vocab_size = 5


model_pms = {"width":10,
             "height":10,
             "word_length": UserSettableParameter('slider', 'Word length', initial_word_length, minimal_word_length, maximal_word_length),
             "vocab_size": UserSettableParameter('slider', 'Vocabulary size', initial_vocab_size, minimal_vocab_size, maximal_vocab_size),
             "initial_degree_of_iconicity": UserSettableParameter('slider', "Initial iconicity %", 60, 0, 100),
             "learning_error_degree": UserSettableParameter('slider', "L2 learning error %", 20, 0, 100),
             "turnover_chance": UserSettableParameter('slider', 'Turnover chance (%)', 50, 0, 100),
             "turnover_threshold": UserSettableParameter('slider', 'Turnover threshold', 10, 5, 50),
             "l2_radius": UserSettableParameter('slider', 'L2 acquisition radius', 100, 1, 100),
             "l2_replace_chance": UserSettableParameter('slider', 'L2 replace chance', 50, 0, 100)}

In [ ]:
chart = ChartModule([#{"Label": "Total avg. iconicity",
                      #"Color": "Black"},
                     {"Label": "L1 avg. iconicity",
                      "Color": "Green"},
                     {"Label": "L2 avg. iconicity",
                      "Color": "Red"},
                     {"Label": "Avg. convergence ratio",
                      "Color": "Blue"}
                    ],
                   data_collector_name="datacollector")

In [ ]:
server = ModularServer(IconicityModel,
                       [grid, chart],
                       "Iconicity Model",
                       model_pms)
server.port = 8521
server.launch()